<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/DistilBert_for_NER_tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 42.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.3/324.3 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
  Attemptin

In [2]:
import pandas as pd
from datasets import load_dataset
data = load_dataset("tomaarsen/conllpp")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [3]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'document_id', 'sentence_id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'document_id', 'sentence_id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'document_id', 'sentence_id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [4]:
data['train'].features

{'id': Value(dtype='string', id=None),
 'document_id': Value(dtype='int32', id=None),
 'sentence_id': Value(dtype='int32', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None),
 'chunk_tags': Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC'

In [5]:
pd.DataFrame(data['train'][:])[['tokens','ner_tags']].iloc[0]

,0
tokens,"[EU, rejects, German, call, to, boycott, Briti..."
ner_tags,"[3, 0, 7, 0, 0, 0, 7, 0, 0]"


In [6]:
tags=data['train'].features['ner_tags'].feature

In [7]:
index2tag={ idx:tag for idx,tag in enumerate(tags.names)}
tag2index={tag:idx for idx,tag in enumerate(tags.names)}

In [8]:
tags.int2str(3)

'B-ORG'

In [9]:
def create_tag_names(batch):
  tag_name={'ner_tags_str':[tags.int2str(idx) for idx in batch['ner_tags']]}
  return tag_name


In [10]:
data=data.map(create_tag_names)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [11]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'document_id', 'sentence_id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_str'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'document_id', 'sentence_id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_str'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'document_id', 'sentence_id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'ner_tags_str'],
        num_rows: 3453
    })
})

In [12]:
pd.DataFrame(data['train'][:])[['tokens','ner_tags','ner_tags_str']].iloc[0]

,0
tokens,"[EU, rejects, German, call, to, boycott, Briti..."
ner_tags,"[3, 0, 7, 0, 0, 0, 7, 0, 0]"
ner_tags_str,"[B-ORG, O, B-MISC, O, O, O, B-MISC, O, O]"


In [13]:
#Model Building
from transformers import AutoTokenizer

model_checkpoint="distilbert-base-cased"

tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
tokenizer.is_fast

True

In [15]:
inputs=[data['train'][0]['tokens']]

In [16]:
inputs=tokenizer(inputs,is_split_into_words=True)
inputs

{'input_ids': [[101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [17]:
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [18]:
data['train'][0]['tokens']

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [19]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [20]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            current_word = word_id
            # Ensure labels is a list or dictionary
            if word_id is None:
                new_labels.append(-100)
            else:
                label = labels[word_id] if word_id < len(labels) else -100
                new_labels.append(label)

        elif word_id is None:
            new_labels.append(-100)
        else:
            # Only execute if word_id is valid
            if word_id < len(labels):
                label = labels[word_id]
                # Ensure label is an integer and perform the operation
                if isinstance(label, int) and label % 2 == 1:
                    label += 1
                new_labels.append(label)
            else:
                new_labels.append(-100)

    return new_labels

In [21]:
labels=data['train'][0]['ner_tags']
word_ids=inputs.word_ids()
print(labels,word_ids)

[3, 0, 7, 0, 0, 0, 7, 0, 0] [None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]


In [22]:
align_labels_with_tokens(labels,word_ids)

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]

In [23]:
def tokenize_and_align_labels(examples):
  tokenized_inputs=tokenizer(examples['tokens'],truncation=True,is_split_into_words=True)

  all_labels=examples['ner_tags']

  new_labels=[]
  for idx,label in enumerate(all_labels):
    word_ids=tokenized_inputs.word_ids(idx)
    new_labels.append(align_labels_with_tokens(label,word_ids))

  tokenized_inputs['labels']=new_labels
  return tokenized_inputs

In [24]:
tokenized_datasets=data.map(tokenize_and_align_labels,batched=True,remove_columns=data['train'].column_names)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [25]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

### Data Collection and Metrics

In [26]:
from transformers import DataCollatorForTokenClassification

data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)

In [27]:
batch=data_collator([tokenized_datasets['train'][i] for i in range(2)])

In [28]:
batch

{'input_ids': tensor([[  101,  7270, 22961,  1528,  1840,  1106, 21423,  1418,  2495, 12913,
           119,   102],
        [  101,  1943, 14428,   102,     0,     0,     0,     0,     0,     0,
             0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]), 'labels': tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])}

In [29]:
!pip install seqeval
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=109f7404f90cf88dec7be7dc4489429036d5600d05119747d2a8f93371226dd9
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00


In [30]:
import evaluate
metrics=evaluate.load('seqeval')

In [31]:
ner_feature=data['train'].features['ner_tags']
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [32]:
label_names=ner_feature.feature.names

In [33]:
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [34]:
labels=data['train'][0]['ner_tags']
labels=[label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [35]:
predictions=labels.copy()
predictions[2]="O"

metrics.compute(predictions=[predictions],references=[labels])


{'MISC': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.8888888888888888}

In [55]:
import numpy as np

def compute_metrics(eval_preds):
  logits,labels=eval_preds

  predictions=np.argmax(logits,axis=-1)

  true_labels=[[label_names[l] for l in label if l != -100] for label in labels ]

  true_predictions=[[ label_names[p] for p,l in zip(prediction,label) if l!=-100]for prediction,label in zip(predictions,labels)]

  all_metrics =metrics.compute(predictions=true_predictions,references=true_labels)

  return {
      'precision':all_metrics['overall_precision'],
      'recall':all_metrics['overall_recall'],
      'f1':all_metrics['overall_f1'],
      'accuracy':all_metrics['overall_accuracy']
  }

In [56]:
id2label={ i:label for i,label in enumerate(label_names)}
label2id={label:i for i,label in enumerate(label_names)}

In [57]:
##Model Training

from transformers import AutoModelForTokenClassification

model=AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,id2label=id2label,
    label2id=label2id
    )

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
print(id2label)

{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-MISC', 8: 'I-MISC'}


In [59]:
model.config.num_labels

9

In [60]:
from transformers import TrainingArguments
args=TrainingArguments(
                  "distilbert-finetuned-ner",
                  evaluation_strategy="epoch",
                  save_strategy="epoch",
                  learning_rate=2e-5,
                  num_train_epochs=3,
                  weight_decay=0.01,
                  push_to_hub=False
                  )

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [61]:
from transformers import Trainer
trainer=Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['validation'],
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.091800,0.086211,0.878199,0.912487,0.895015,0.975776
2,0.045700,0.074685,0.910976,0.931673,0.921208,0.981456
3,0.029800,0.071927,0.917023,0.939246,0.928001,0.983370


TrainOutput(global_step=5268, training_loss=0.08144185736764084, metrics={'train_runtime': 384.3186, 'train_samples_per_second': 109.604, 'train_steps_per_second': 13.707, 'total_flos': 460431563935266.0, 'train_loss': 0.08144185736764084, 'epoch': 3.0})

In [63]:
from transformers import pipeline
checkpoint="/content/distilbert-finetuned-ner/checkpoint-5268"

token_classifier=pipeline(
    "token-classification",model=checkpoint,aggregation_strategy="simple"
)

token_classifier("My name is S.Praveen Kumar. I work at JNTU and live in Hyderabad")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity_group': 'PER',
  'score': 0.99675936,
  'word': 'S. Praveen Kumar',
  'start': 11,
  'end': 26},
 {'entity_group': 'ORG',
  'score': 0.99827105,
  'word': 'JNTU',
  'start': 38,
  'end': 42},
 {'entity_group': 'LOC',
  'score': 0.9987633,
  'word': 'Hyderabad',
  'start': 55,
  'end': 64}]

In [68]:
!zip -r distilbert-finetuned-ner.zip /content/distilbert-finetuned-ner/checkpoint-5268

  adding: content/distilbert-finetuned-ner/checkpoint-5268/ (stored 0%)
  adding: content/distilbert-finetuned-ner/checkpoint-5268/scheduler.pt (deflated 56%)
  adding: content/distilbert-finetuned-ner/checkpoint-5268/model.safetensors (deflated 8%)
  adding: content/distilbert-finetuned-ner/checkpoint-5268/training_args.bin (deflated 51%)
  adding: content/distilbert-finetuned-ner/checkpoint-5268/vocab.txt (deflated 49%)
  adding: content/distilbert-finetuned-ner/checkpoint-5268/trainer_state.json (deflated 68%)
  adding: content/distilbert-finetuned-ner/checkpoint-5268/optimizer.pt (deflated 25%)
  adding: content/distilbert-finetuned-ner/checkpoint-5268/tokenizer_config.json (deflated 76%)
  adding: content/distilbert-finetuned-ner/checkpoint-5268/rng_state.pth (deflated 25%)
  adding: content/distilbert-finetuned-ner/checkpoint-5268/special_tokens_map.json (deflated 42%)
  adding: content/distilbert-finetuned-ner/checkpoint-5268/tokenizer.json (deflated 70%)
  adding: content/disti